<a href="https://colab.research.google.com/github/jcandane/StochasticPhysics/blob/main/instrument.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import stuff

Also include default test cases

In [ ]:
import numpy as np
import gpcam
from gpcam import AutonomousExperimenterGP
import time
import h5py ## https://docs.h5py.org/en/stable/quick.html
import datetime

data_labels=["x_data", "y_data", "noise variance", "cost", "id", "time stamp", "date time", "measured"]
list_of_data=[{'x_data': np.array([3.04986762, 4.66446892]),
  'y_data': -0.6519311743614136,
  'noise variance': None,
  'cost': [np.array([0, 0]), np.array([3.04986762, 4.66446892]), 7.714336536446131],
  'id': '3b115847-cbe5-4c19-b2c9-0004455f8a52',
  'time stamp': 1705512086.8447511,
  'date time': '17/01/2024_11:21:26',
  'measured': False},
 {'x_data': np.array([1.07469298, 6.04996553]),
  'y_data': -0.13806646827231991,
  'noise variance': None,
  'cost': [np.array([0, 0]), np.array([1.07469298, 6.04996553]), 7.124658510619914],
  'id': 'e343a7e8-b133-415e-a69f-6768438e6934',
  'time stamp': 1705512086.844794,
  'date time': '17/01/2024_11:21:26',
  'measured': False},
 {'x_data': np.array([4.54385235, 6.1862373 ]),
  'y_data': 0.9841470549061848,
  'noise variance': None,
  'cost': [np.array([0, 0]), np.array([4.54385235, 6.1862373 ]), 10.730089647376804],
  'id': '1e7a8bc2-9e1e-4f92-b5e0-f8205b63f259',
  'time stamp': 1705512086.844807,
  'date time': '17/01/2024_11:21:26',
  'measured': False},
 {'x_data': np.array([0.36948575, 7.26536243]), "y_data": None }]


def instrument0(data):
    """
    GIVEN : data (List[dic]) gpCAM only gives the data it is using at-the-moment
    """
    for entry in data:
        entry["y_data"] = np.sin(np.linalg.norm(entry["x_data"]))
        entry["cost"]   = [np.array([0,0]),entry["x_data"],np.sum(entry["x_data"])]
    return data

#### GP quick-test???
START=time.time()
my_ae = AutonomousExperimenterGP(np.array([[0,10],[0,10]]),
                                 np.ones((3)),
                                 np.array([[0.001,100.],[0.001,100.],[0.001,100.]]), ## limits
                                 init_dataset_size= 20,
                                 instrument_function = instrument0,
                                 cost_function_parameters={"offset": 5.0,"slope":10.0},
                                 store_inv = True)
my_ae.train(method = "hgdl")      ### trains the kernel, to find optimial hyperparameters given the intiially-random data
my_ae.go(len(my_ae.x_data) + 10)  ### runs the Autonomous Experiment
print(time.time()-START)

3.182755708694458


In [ ]:
######
def math_function(x_data):
    return np.sin(np.linalg.norm(x_data, axis=1))


def gpcam_to_h5(data, filename="to_vintrument.h5"):
    """
    this function reads gpcam's data, and creates an h5 file (to be read by the intrument)
    GIVEN : data : List[dic]
    """

    to_analyze=[]
    for entry in data:
        to_analyze.append(entry["x_data"])
    to_analyze = np.asarray(to_analyze) ## make into a np.array, D_ax

    h5f = h5py.File(filename, "w")
    h5f.create_dataset("dataset_1", data=to_analyze)
    h5f.close()
    return None

def h5_to_vinstrument(filename="to_vintrument.h5"):
    """
    this function reads a h5 file, to obtain a 2d-numpy.array (to be used by the virtual-intrument)
    GIVEN :
    GET   :
    """

    h5f    = h5py.File(filename, "r")
    x_data = np.asarray(h5f.get('dataset_1'))
    h5f.close()

    return x_data ### numpy.array of dimensions ( samples , coordinates ) i.e. D_ax

def vinstrument_to_h5(y_data, filename="from_vintrument.h5"):
    """
    this function obtained the vintrument's y_data, along with other meta-data saves to an h5
    GIVEN : y_data (D_aX)
    """

    h5f = h5py.File(filename, "w")
    h5f.create_dataset("dataset_1", data=y_data)
    h5f.close()
    return None

def h5_to_gpcam(data, filename="from_vintrument.h5"):
    """
    this function updates gpcam's 'data' variable (List[dic]), by reading a h5 file.
    """
    h5f    = h5py.File(filename, "r")
    y_data = np.asarray(h5f["dataset_1"]) ## D_aX
    h5f.close()

    for a, entry in enumerate(data):
        entry["y_data"] = y_data[a]

    return data

def vinstrument():

    x_data = h5_to_vinstrument()

    y_data = math_function(x_data)

    vinstrument_to_h5(y_data)

    return None

def instrument(data):

    ### gpcam -> h5 (x-coordinates only)
    gpcam_to_h5(data)

    ### vintrument()
    vinstrument()

    ### h5 -> gpcam (everything)
    data = h5_to_gpcam(data)

    return data

## gpCAM run with $\texttt{vinstrument}$

In [ ]:
#### GP quick-test???
START=time.time()
my_ae = AutonomousExperimenterGP(np.array([[0,10],[0,10]]),
                                 np.ones((3)),
                                 np.array([[0.001,100.],[0.001,100.],[0.001,100.]]), ## limits
                                 init_dataset_size= 20,
                                 instrument_function = instrument,
                                 cost_function_parameters={"offset": 5.0,"slope":10.0},
                                 store_inv = True)
my_ae.train(method = "hgdl")      ### trains the kernel, to find optimial hyperparameters given the intiially-random data
my_ae.go(len(my_ae.x_data) + 10)  ### runs the Autonomous Experiment
print(time.time()-START)

2.59155011177063
